In [6]:
import numpy as np
import pandas as pd
from pqp.data.data import Data
from pqp.estimation.model import Model
from pqp.causal.graph import Graph
from pqp.parametric.categorical_distribution import CategoricalDistribution
from pqp.causal.estimands import *
from pqp.symbols import *

In [7]:
def foo(**kwargs):
    print(kwargs)
    print(kwargs["bar"])

foo(bar=1)
foo()

{'bar': 1}
1
{}


KeyError: 'bar'

In [5]:
    x, y, z = make_vars("xyz")
ATE(y, {x: 1}, {x: 0}).expression().display()
(Expectation(y, P(y, given=do(x.val == 1))) - Expectation(y, P(y, given=[do(x.val == 0)]))).display()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [2]:
severity = np.array([0, 1])
treatment = severity                            # perfect doctor
pathway = (1-treatment)*0.3                     # treatment => 0, control => 0.5
outcome = (severity + pathway)                  # higher is worse

df = pd.DataFrame({
    "severity": severity,
    "treatment": treatment,
    "pathway": pathway,
    "outcome": outcome,
})

data = Data(df, {
    "severity": "discrete",
    "treatment": "discrete",
    "pathway": "discrete",
    "outcome": "discrete",
})

v = data.vars

In [6]:
god_model = Model(
    CategoricalDistribution(data, prior=0.1),
    Graph([
        v.outcome <= [v.pathway, v.severity],
        v.pathway <= v.treatment,
        v.treatment <= v.severity,
    ])
)
est = god_model.estimand("outcome", do=["treatment"])
print("god interventional")
est.display()

god interventional


<IPython.core.display.Math object>

In [11]:
np.random.seed(137)
n = 5

# severity = np.random.binomial(3, 0.5, size=n)
# doctor_noise = np.random.binomial(1, 0.1, size=n)
# print((severity > 1).astype(int))
# print(doctor_noise)
# treatment = ((severity > 1) != doctor_noise).astype(int)

# # print(treatment)

# pathway_predisposition = 0# np.random.binomial(2, 0.5, size=n)
# pathway = pathway_predisposition - treatment

# outcome_noise = 0#np.random.binomial(2, 0.5, size=n)
# outcome = pathway + severity + outcome_noise

severity = np.random.binomial(1, 0.5, size=n)
treatment = severity                            # perfect doctor
pathway = (1-treatment)*0.5                     # treatment => 0, control => 0.5
outcome = 1 - (severity + pathway)              # higher is worse

df = pd.DataFrame({
    "severity": severity,
    "treatment": treatment,
    "pathway": pathway,
    "outcome": outcome,
})

df.head(10)

,severity,treatment,pathway,outcome
0,1,1,0.0,0.0
1,0,0,0.5,0.5
2,1,1,0.0,0.0
3,1,1,0.0,0.0
4,0,0,0.5,0.5


In [12]:
data = Data(df, {
    "severity": "binary",
    "treatment": "binary",
    "pathway": "discrete",
    "outcome": "discrete",
})

dist = CategoricalDistribution(data, prior=1)

In [13]:
naive_graph = Graph([data.vars.outcome <= data.vars.treatment])
naive_model = Model(dist, naive_graph)

naive_model.expectation("outcome", do={"treatment": 1})

0.03571428571428571

In [14]:
graph = Graph([
    data.vars.outcome <= data.vars.pathway,
    data.vars.pathway <= data.vars.treatment,
    data.vars.treatment & data.vars.outcome,
])

model = Model(dist, graph)


$$
\sum_{m} \big({\sum_{x} \big({P(x) P(x, m, y) \over P(x, m)}\big) P(x, m) \over P(x)}\big)
$$

$$
\big({\sum_{x} \big({P(x) P(x, m=0, y) \over P(x, m=0)}\big) P(x, m=0) \over P(x)}\big) + \big({\sum_{x} \big({P(x) P(x, m=1, y) \over P(x, m=1)}\big) P(x, m=1) \over P(x)}\big)
$$

$$
P(m=0 | x)\big({P(x=0) P(x=0, m=0, y) \over P(x=0, m=0)} + {P(x=1) P(x=1, m=0, y) \over P(x=1, m=0)}\big) +\\
P(m=1 | x)\big({P(x=0) P(x=0, m=1, y) \over P(x=0, m=1)} + {P(x=1) P(x=1, m=1, y) \over P(x=1, m=1)}\big)
$$

$$
P(m=0 | x)P(x=0) P(y | x=0, m=0) + P(m=0 | x)P(x=1) P(y | x=1, m=0) +\\
P(m=1 | x)P(x=0) P(y | x=0, m=1) + P(m=1 | x)P(x=1) P(y  | x=1, m=1)
$$

$$
P(y=0 | do(x = 1)) =
P(m=0 | x=1)P(x=0) P(y=0 | x=0, m=0) + P(m=0 | x=1)P(x=1) P(y | x=1, m=0) +\\
P(m=1 | x=1)P(x=0) P(y=0 | x=0, m=1) + P(m=1 | x=1)P(x=1) P(y | x=1, m=1)
$$

$$
P(y = 1 | do(x = 1)) = \\
P(m=0 | x= 1)P(x=0) P(y= 1 | x=0, m=0) + P(m=0 | x= 1)P(x=1) P(y= 1 | x=1, m=0) +\\
P(m=1 | x= 1)P(x=0) P(y= 1 | x=0, m=1) + P(m=1 | x= 1)P(x=1) P(y= 1  | x=1, m=1)
$$

x | m | y | P
---|---|---|---
0 | 0 | 0 | $\epsilon$
0 | 0 | 1 | $\epsilon$
0 | 1 | 0 | $\epsilon$
0 | 1 | 1 | 0.5
1 | 0 | 0 | $\epsilon$
1 | 0 | 1 | 0.5
1 | 1 | 0 | $\epsilon$
1 | 1 | 1 | $\epsilon$

$$
P(y = 1 | do(x = 1)) = 
1 * 1 * 0.5 + 0 * 0.5 * 1 +\\
0 * 0.5 * 1 + 0 * 0.5 * 0.5
$$

$$
P(y = 1 | do(x = 1)) = 
0.5
$$

$$
P(y=0 | do(x = 1)) =
1 * 0.5 * 0.5 + 0 * 0.5 * 0 +\\
0 * 0.5 * 0 + 0 * 0.5 * 0.5
$$

$$
P(y=0 | do(x = 1)) = 0.25
$$